This notebook contains the code to write for each step of the project. 

In [7]:
#Visualizing the video
import cv2
video_path = 'data\data_v\parking_crop_loop.mp4'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open video source.")
    exit()

ret=True
while ret:
    ret, frame= cap.read()
    
    if not ret:
        print("Error: Could not read frame.")
        break
    
    cv2.imshow('Video Frame', frame)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

: 

In [ ]:
#load the mask 
import cv2
mask = 'mask\mask_crop.png'
mask = cv2.imread(mask, 0)

In [ ]:
#detect the parking spots and create the bounding boxes using the mask
from utils import get_parking_spots_boxes
connected_componets = cv2.connectedComponentsWithStats(mask, 4, cv2.CV_32S)
parking_spots = get_parking_spots_boxes(connected_componets)
print(parking_spots[0]) #getting the value of the coordonnées of the first parking spot

[70, 0, 131, 56]


In [ ]:
#adding to the video the bounding boxes in Blue
for spot in parking_spots:
        x1,y1, w, h = spot
        frame = cv2.rectangle(frame, (x1,y1), (x1+w,y1+h), (255,0,0))

In [ ]:
#classifying each blue bounding box to know if it is available or not 
#Red => Not available, Green => Available